## FT084 - Introdução a Mineração de Dados
---
### Tarefa 03: Multilayer Perceptrons (MLP)

Este código tem por objetivo a resolução da tarefa em questão, que consiste na implementação de redes neurais do tipo MLP.  
Instruções para o experimento:
1. Utilize a mesma metodologia da atividade anterior, ou seja, subamostragem aleatória com 5 repetições, sendo 70% dos dados para treinamento e 30% dos dados para teste (em cada repetição);
2. Faça todos os ajustes necessários no conjunto de dados para que as MLPs possam ser aplicadas (e descreva, no relatório, os ajustes feitos);
3. Apresente o erro médio de classificação, para o conjunto de testes, e compare os resultados com os obtidos na atividade anterior;
4. Apresente claramente a metodologia adotada, os classificadores utilizados (versões e parâmetros) e discuta os resultados obtidos.
---

#### 1) Importação das bibliotecas  
Serão utilizados alguns pacotes para a implementação do código. São eles:
- pandas: leitura dos arquivos
- numpy, scipy: cálculo de algumas estatísticas
- sklearn: separação dos dados entre treino e teste, transformação dos atributos categóricos para numéricos (caso necessário), matriz de confusão e avaliação do erro
- keras: algoritmo de classificação

In [1]:
# Importação das Bibliotecas
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils

Using TensorFlow backend.


#### 2) Criação do dataset
Será gerado um objeto do tipo dataframe com a base de dados bupa.data, utilizando os nomes dos atributos disposíveis em bupa.names.

In [2]:
# Nomes das colunas
columns_names = [
    'mcv',
    'alkphos',
    'sgpt',
    'sgot',
    'gammagt',
    'drinks',
    'selector'
]

# Criação do dataset
dataset = pd.read_table("bupa.data", sep = ",", header = None, names = columns_names)
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
...,...,...,...,...,...,...,...
340,99,75,26,24,41,12.0,1
341,96,69,53,43,203,12.0,2
342,98,77,55,35,89,15.0,1
343,91,68,27,26,14,16.0,1


#### 3) Análise da base de dados
Analisar o tipo dos dados da base (numéricos ou categóricos), se há valores faltantes, e se é necessário realizar alguma transformação prévia.

In [3]:
# Observando o tamanho da base de dados
dataset.shape

(345, 7)

In [4]:
# Utilizando o método describe() para analisar a base de dados e validar se todos os atributos são numéricos
dataset.describe()

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
count,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000,345.000000
mean,90.159420,69.869565,30.405797,24.643478,38.284058,3.455072,1.579710
std,4.448096,18.347670,19.512309,10.064494,39.254616,3.337835,0.494322
min,65.000000,23.000000,4.000000,5.000000,5.000000,0.000000,1.000000
25%,87.000000,57.000000,19.000000,19.000000,15.000000,0.500000,1.000000
50%,90.000000,67.000000,26.000000,23.000000,25.000000,3.000000,2.000000
75%,93.000000,80.000000,34.000000,27.000000,46.000000,6.000000,2.000000
max,103.000000,138.000000,155.000000,82.000000,297.000000,20.000000,2.000000


In [5]:
# Criando um vetor para verificar se há valores nulos
null_array = [dataset.iloc[:, i].isnull().unique() for i in range(7)]
null_array

[array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False])]

In [6]:
# Criando um vetor para verificar se há valores faltantes
na_array = [dataset.iloc[:, i].isna().unique() for i in range(7)]
na_array

[array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False]),
 array([False])]

#### 4) Transformação da base de dados
Classificar a coluna "drinks" de acordo com as informações do arquivo bupa.names, além de analisar a coluna "selector", avaliando se pode ser usada como divisor dos dados.
- "drinks" 3 5 pode ser considerado um tipo de seleção na base de dados (de acordo com esse artigo: McDermott & Forsyth 2016, Diagnosing a disorder in a classification benchmark, Pattern Recognition Letters, Volume 73.)
- "selector" divide a base de dados em uma determina proporção; será avaliada essa proporção, e se ela é satisfatória

In [7]:
# Classificando a coluna "drinks"
dataset.loc[dataset['drinks'] <= 6, 'drinks'] = 0
dataset.loc[dataset['drinks'] > 6, 'drinks'] = 1
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks,selector
0,85,92,45,27,31,0.0,1
1,85,64,59,32,23,0.0,2
2,86,54,33,16,54,0.0,2
3,91,78,34,24,36,0.0,2
4,87,70,12,28,10,0.0,2
...,...,...,...,...,...,...,...
340,99,75,26,24,41,1.0,1
341,96,69,53,43,203,1.0,2
342,98,77,55,35,89,1.0,1
343,91,68,27,26,14,1.0,1


In [8]:
# Como a proporção é diferente de 30% e 70%, a coluna "selector" será descartada do dataset.
dataset.drop(columns = 'selector', inplace = True)
columns_names.pop(-1)
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,85,92,45,27,31,0.0
1,85,64,59,32,23,0.0
2,86,54,33,16,54,0.0
3,91,78,34,24,36,0.0
4,87,70,12,28,10,0.0
...,...,...,...,...,...,...
340,99,75,26,24,41,1.0
341,96,69,53,43,203,1.0
342,98,77,55,35,89,1.0
343,91,68,27,26,14,1.0


#### 5) Normalização dos Dados
Procedimento realizado antes de iniciar a construção do modelo, a fim de todos os valores estarem na mesma escala.

In [9]:
numpy_array = dataset.values
min_max_scaler = preprocessing.MinMaxScaler()
new_dataset = min_max_scaler.fit_transform(numpy_array)
dataset = pd.DataFrame(new_dataset, columns = columns_names)
dataset

,mcv,alkphos,sgpt,sgot,gammagt,drinks
0,0.526316,0.600000,0.271523,0.285714,0.089041,0.0
1,0.526316,0.356522,0.364238,0.350649,0.061644,0.0
2,0.552632,0.269565,0.192053,0.142857,0.167808,0.0
3,0.684211,0.478261,0.198675,0.246753,0.106164,0.0
4,0.578947,0.408696,0.052980,0.298701,0.017123,0.0
...,...,...,...,...,...,...
340,0.894737,0.452174,0.145695,0.246753,0.123288,1.0
341,0.815789,0.400000,0.324503,0.493506,0.678082,1.0
342,0.868421,0.469565,0.337748,0.389610,0.287671,1.0
343,0.684211,0.391304,0.152318,0.272727,0.030822,1.0


#### 6) Divisão entre Classe e Atributos
Definir dentro do dataset qual é a variável que será classificada, e quais são as variáveis preditoras.

In [10]:
# Definição das Classes
classes = dataset['drinks']

# Definição dos atributos
attributes_names = [
    'mcv',
    'alkphos',
    'sgpt',
    'sgot',
    'gammagt'
]

attributes = dataset[attributes_names]

# Transformação da classe para o formato "dummy", pois temos uma rede neural com 3 neurônios na camada de saída
class_dummy = np_utils.to_categorical(classes)

#### 7) Criação do Modelo
Criaçao de uma função que irá encapsular o processo de criação da rede neural, retornando a classe prevista para cada instância e o erro de classificação do algoritmo.

In [11]:
def mlp():
    # Divisão da base de dados entre treinamento e teste.
    x_train, x_test, y_train, y_test = train_test_split(attributes, class_dummy, test_size = 0.3, random_state = 0)
    
    # Criação da estrutura da rede neural com a classe Sequential (sequência de camadas)
    modelo = Sequential()
    modelo.add(Dense(units = 100, input_dim = 5))
    # modelo.add(Dense(units = 50))
    modelo.add(Dense(units = 2, activation = 'softmax'))
    
    # Visualização da estrutura da rede neural
    # modelo.summary()
    
    # Configuração dos parâmetros da rede neural (adam = algoritmo para atualizar os pesos e loss = cálculo do erro)
    modelo.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    # Treinamento, dividindo a base de treinamento em uma porção para validação (validation_data)
    modelo.fit(x_train, y_train, epochs = 100, validation_data = (x_test, y_test))
    
    # Previsões
    previsoes = modelo.predict(x_test)
    previsoes = (previsoes > 0.5)
    previsoes_final = [0 if i[0] == True else 1 for i in previsoes]
    
    # Precisão e Erro
    y_test_final = [0 if i[0] == 1 else 1 for i in y_test]
    precisao = accuracy_score(y_test_final, previsoes_final)
    precisao
    erro = 1 - precisao
    
    
    return erro

In [12]:
mlp()

Train on 241 samples, validate on 104 samples
Epoch 1/100
241/241 [==============================] - 0s 306us/step - loss: 0.6963 - accuracy: 0.4813 - val_loss: 0.5938 - val_accuracy: 0.9038
Epoch 2/100
241/241 [==============================] - 0s 29us/step - loss: 0.5638 - accuracy: 0.8423 - val_loss: 0.4796 - val_accuracy: 0.9038
Epoch 3/100
241/241 [==============================] - 0s 29us/step - loss: 0.5018 - accuracy: 0.8423 - val_loss: 0.4124 - val_accuracy: 0.9038
Epoch 4/100
241/241 [==============================] - 0s 25us/step - loss: 0.4698 - accuracy: 0.8423 - val_loss: 0.3823 - val_accuracy: 0.9038
Epoch 5/100
241/241 [==============================] - 0s 29us/step - loss: 0.4610 - accuracy: 0.8423 - val_loss: 0.3695 - val_accuracy: 0.9038
Epoch 6/100
241/241 [==============================] - 0s 25us/step - loss: 0.4571 - accuracy: 0.8423 - val_loss: 0.3645 - val_accuracy: 0.9038
Epoch 7/100
241/241 [==============================] - 0s 25us/step - loss: 0.4543 - accu

0.11538461538461542